---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
format1 = r"(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})"
#format2 = r"((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)*[a-z]?[-. ](\d{1,2})[,]?[- ](\d{4}))"
format2 = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.]?[- ](\d{1,2})[,]?[- ](\d{4})'
format3 = r"(\d{1,2})[ ]((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[,.]? (\d{2,4})"
#format3 = r'(\d{1,2}) ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[.,]? (\d{4})'
format4 = r"(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ ](\d{1,2})[a-z]{2}[,](\d{2,4})"
#format5 = r"(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ ](\d{4})"
format5 = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[,]? (\d{4})'

format6 = r"(\d{1,2})[/](\d{4})"
format7 = r"(\d{4})"

In [3]:
# Receibes the abreviated version of a month and return the numerical string version
def decodeAbbMonth(month):
    monthsEncode =  {'Jan': '01', 'Feb': '02' ,'Mar': '03','Apr':'04','May':'05','Jun':'06',
                      'Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}
    if(month in monthsEncode.keys()):
        return monthsEncode[month]
    else:
        return monthEnconde["Jan"]

In [4]:
def decodeLargeMonth(month):
    monthsEncode =  {'January': '01', 'February': '02' ,'March': '03','April':'04','May':'05','June':'06',
                      'July':'07','August':'08','September':'09','October':'10','November':'11','December':'12'}
    if(month in monthsEncode.keys()):
        return monthsEncode[month]
    else:
        return monthsEncode["January"]

In [5]:
def date_sorter():
    count = 0
    normalizedDates = list()

    for medNote in df:
        count += 1
        if(len(re.findall(format1,medNote))>0):
            #date = list(re.findall(format1,medNote))
            # Normalize date 
            date = list(re.findall(format1,medNote)[0])
            if (len(date[2]) == 2):
                date[2] = "19"+date[2]
            if len(date[0])==1:
                date[0] = '0'+ date[0]
            if len(date[1])==1:
                date[1] = '0'+ date[1]
            normalizedDates.append(date[2]+date[0]+date[1])
        elif(len(re.findall(format2,medNote))>0):
            date = list(re.findall(format2,medNote)[0])
            if (len(date[1]) == 1):
                date[1] = "0"+ date[1]
            normalizedDates.append(date[2]+decodeAbbMonth(date[0])+date[1])
        elif(len(re.findall(format3,medNote))>0):
            date = list(re.findall(format3,medNote)[0])
            normalizedDates.append(date[2]+decodeAbbMonth(date[1])+date[0])
        elif(len(re.findall(format4,medNote))>0):
            print("format 4:")
            date = list(re.findall(format4,medNote)[0])
            normalizedDates.append(date[2]+decodeAbbMonth(date[0])+date[1][:3])
        elif(len(re.findall(format5,medNote))>0):
            date = list(re.findall(format5,medNote)[0])
            normalizedDates.append(date[1]+decodeAbbMonth(date[0])+"01")
        elif(len(re.findall(format6,medNote))>0):
            date = list(re.findall(format6,medNote)[0])
            if(len(date[0]) == 1):
                date[0] = "0" + date[0]
            normalizedDates.append(date[1]+ date[0]+"01")

        elif(len(re.findall(format7,medNote))>0):
            date = list(re.findall(format7,medNote))
            normalizedDates.append(date[0]+"01"+"01")

            
    years = list()
    months = list()
    days = list()
    for date in normalizedDates:
        years.append(date[:4])
        months.append(date[4:6])
        days.append(date[6:])
        
    dates = pd.DataFrame({"Year":years,"Month":months,"Day":days})
    dates.sort_values(by=['Year',"Month","Day"],inplace = True)
    return dates.index

In [6]:
date_sorter()

Int64Index([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98,
            ...
            152, 235, 464, 253, 231, 427, 141, 186, 161, 413],
           dtype='int64', length=500)